# Comprehensive Stock Market Analysis with yfinance and Polars

This notebook demonstrates extensive multi-timeframe analysis of stock market data using:
- **yfinance**: For fetching real-time and historical stock data
- **Polars**: For high-performance data manipulation and analysis

## Timeframes Covered:
- 10-minute intervals
- 15-minute intervals
- 30-minute intervals
- 1-hour intervals
- 4-hour intervals
- Daily intervals

## Analysis Includes:
- Price action analysis
- Volume analysis
- Technical indicators (SMA, EMA, RSI, MACD, Bollinger Bands)
- Volatility metrics
- Support/Resistance levels
- Cross-timeframe analysis

In [ ]:
# Import required libraries
import polars as pl
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configure Polars display
pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(15)

## 1. Data Acquisition from yfinance

We'll fetch stock data for multiple tickers across different timeframes.

In [ ]:
# Define the stock tickers to analyze
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
PRIMARY_TICKER = 'AAPL'  # Primary ticker for detailed analysis

# Define timeframe configurations
TIMEFRAMES = {
    '10min': {'interval': '10m', 'period': '5d'},
    '15min': {'interval': '15m', 'period': '5d'},
    '30min': {'interval': '30m', 'period': '1mo'},
    '1hour': {'interval': '1h', 'period': '3mo'},
    '4hour': {'interval': '1h', 'period': '1y'},  # We'll resample 1h to 4h
    '1day': {'interval': '1d', 'period': '2y'}
}

print("Timeframes configured for analysis:")
for tf, config in TIMEFRAMES.items():
    print(f"  {tf}: interval={config['interval']}, period={config['period']}")

In [ ]:
def fetch_stock_data(ticker: str, interval: str, period: str) -> pl.DataFrame:
    """
    Fetch stock data from yfinance and convert to Polars DataFrame.
    
    Args:
        ticker: Stock ticker symbol
        interval: Data interval (1m, 5m, 15m, 30m, 1h, 1d, etc.)
        period: Data period (1d, 5d, 1mo, 3mo, 1y, 2y, etc.)
    
    Returns:
        Polars DataFrame with stock data
    """
    print(f"Fetching {ticker} data: interval={interval}, period={period}")
    
    # Download data from yfinance
    stock = yf.Ticker(ticker)
    df_pandas = stock.history(period=period, interval=interval)
    
    if df_pandas.empty:
        print(f"Warning: No data returned for {ticker}")
        return pl.DataFrame()
    
    # Reset index to make datetime a column
    df_pandas = df_pandas.reset_index()
    
    # Convert to Polars
    df = pl.from_pandas(df_pandas)
    
    # Rename columns to lowercase for consistency
    df = df.rename({
        col: col.lower().replace(' ', '_') 
        for col in df.columns
    })
    
    # Ensure datetime column is properly named
    if 'date' in df.columns:
        df = df.rename({'date': 'datetime'})
    elif 'datetime' not in df.columns and len(df.columns) > 0:
        # First column is usually the datetime
        first_col = df.columns[0]
        if df[first_col].dtype in [pl.Datetime, pl.Date]:
            df = df.rename({first_col: 'datetime'})
    
    # Add ticker column
    df = df.with_columns(pl.lit(ticker).alias('ticker'))
    
    print(f"  Retrieved {len(df)} rows")
    return df

# Test fetch for primary ticker
test_df = fetch_stock_data(PRIMARY_TICKER, '1d', '5d')
print("\nSample data:")
test_df.head()

## 2. Fetch Data for All Timeframes

Download stock data for all configured timeframes.

In [ ]:
# Dictionary to store dataframes for each timeframe
timeframe_data = {}

print("Fetching data for all timeframes...\n")

for tf_name, config in TIMEFRAMES.items():
    print(f"\n{'='*60}")
    print(f"Fetching {tf_name} data")
    print(f"{'='*60}")
    
    df = fetch_stock_data(
        PRIMARY_TICKER,
        config['interval'],
        config['period']
    )
    
    # Handle 4-hour timeframe by resampling 1-hour data
    if tf_name == '4hour' and not df.is_empty():
        print("Resampling 1-hour data to 4-hour intervals...")
        df = df.sort('datetime')
        df = df.with_columns(
            pl.col('datetime').dt.truncate('4h').alias('datetime_4h')
        ).group_by('datetime_4h', 'ticker').agg([
            pl.col('open').first().alias('open'),
            pl.col('high').max().alias('high'),
            pl.col('low').min().alias('low'),
            pl.col('close').last().alias('close'),
            pl.col('volume').sum().alias('volume')
        ]).rename({'datetime_4h': 'datetime'}).sort('datetime')
    
    timeframe_data[tf_name] = df
    print(f"Stored {len(df)} rows for {tf_name}")

print("\n" + "="*60)
print("Data fetch complete!")
print("="*60)

## 3. Technical Indicators Functions

Define functions to calculate various technical indicators using Polars expressions.

In [ ]:
def add_moving_averages(df: pl.DataFrame, periods: list = [5, 10, 20, 50, 200]) -> pl.DataFrame:
    """
    Add Simple Moving Averages (SMA) and Exponential Moving Averages (EMA).
    """
    for period in periods:
        if len(df) >= period:
            df = df.with_columns([
                pl.col('close').rolling_mean(window_size=period).alias(f'sma_{period}'),
                pl.col('close').ewm_mean(span=period).alias(f'ema_{period}')
            ])
    return df


def add_rsi(df: pl.DataFrame, period: int = 14) -> pl.DataFrame:
    """
    Calculate Relative Strength Index (RSI).
    """
    if len(df) < period + 1:
        return df.with_columns(pl.lit(None).alias('rsi'))
    
    # Calculate price changes
    df = df.with_columns(
        (pl.col('close') - pl.col('close').shift(1)).alias('price_change')
    )
    
    # Separate gains and losses
    df = df.with_columns([
        pl.when(pl.col('price_change') > 0)
          .then(pl.col('price_change'))
          .otherwise(0)
          .alias('gain'),
        pl.when(pl.col('price_change') < 0)
          .then(pl.col('price_change').abs())
          .otherwise(0)
          .alias('loss')
    ])
    
    # Calculate average gain and loss
    df = df.with_columns([
        pl.col('gain').ewm_mean(span=period).alias('avg_gain'),
        pl.col('loss').ewm_mean(span=period).alias('avg_loss')
    ])
    
    # Calculate RSI
    df = df.with_columns(
        pl.when(pl.col('avg_loss') == 0)
          .then(100)
          .otherwise(
              100 - (100 / (1 + pl.col('avg_gain') / pl.col('avg_loss')))
          )
          .alias('rsi')
    )
    
    # Clean up intermediate columns
    return df.drop(['price_change', 'gain', 'loss', 'avg_gain', 'avg_loss'])


def add_macd(df: pl.DataFrame, fast: int = 12, slow: int = 26, signal: int = 9) -> pl.DataFrame:
    """
    Calculate MACD (Moving Average Convergence Divergence).
    """
    if len(df) < slow:
        return df.with_columns([
            pl.lit(None).alias('macd'),
            pl.lit(None).alias('macd_signal'),
            pl.lit(None).alias('macd_histogram')
        ])
    
    df = df.with_columns([
        pl.col('close').ewm_mean(span=fast).alias('ema_fast'),
        pl.col('close').ewm_mean(span=slow).alias('ema_slow')
    ])
    
    df = df.with_columns(
        (pl.col('ema_fast') - pl.col('ema_slow')).alias('macd')
    )
    
    df = df.with_columns(
        pl.col('macd').ewm_mean(span=signal).alias('macd_signal')
    )
    
    df = df.with_columns(
        (pl.col('macd') - pl.col('macd_signal')).alias('macd_histogram')
    )
    
    return df.drop(['ema_fast', 'ema_slow'])


def add_bollinger_bands(df: pl.DataFrame, period: int = 20, std_dev: float = 2.0) -> pl.DataFrame:
    """
    Calculate Bollinger Bands.
    """
    if len(df) < period:
        return df.with_columns([
            pl.lit(None).alias('bb_middle'),
            pl.lit(None).alias('bb_upper'),
            pl.lit(None).alias('bb_lower'),
            pl.lit(None).alias('bb_width')
        ])
    
    df = df.with_columns([
        pl.col('close').rolling_mean(window_size=period).alias('bb_middle'),
        pl.col('close').rolling_std(window_size=period).alias('bb_std')
    ])
    
    df = df.with_columns([
        (pl.col('bb_middle') + std_dev * pl.col('bb_std')).alias('bb_upper'),
        (pl.col('bb_middle') - std_dev * pl.col('bb_std')).alias('bb_lower')
    ])
    
    df = df.with_columns(
        (pl.col('bb_upper') - pl.col('bb_lower')).alias('bb_width')
    )
    
    return df.drop('bb_std')


def add_volatility_metrics(df: pl.DataFrame, period: int = 14) -> pl.DataFrame:
    """
    Calculate volatility metrics including ATR (Average True Range).
    """
    if len(df) < 2:
        return df.with_columns([
            pl.lit(None).alias('atr'),
            pl.lit(None).alias('volatility')
        ])
    
    # Calculate True Range
    df = df.with_columns([
        (pl.col('high') - pl.col('low')).alias('hl'),
        (pl.col('high') - pl.col('close').shift(1)).abs().alias('hc'),
        (pl.col('low') - pl.col('close').shift(1)).abs().alias('lc')
    ])
    
    df = df.with_columns(
        pl.max_horizontal('hl', 'hc', 'lc').alias('true_range')
    )
    
    # Calculate ATR
    if len(df) >= period:
        df = df.with_columns(
            pl.col('true_range').rolling_mean(window_size=period).alias('atr')
        )
    
    # Calculate price volatility (standard deviation of returns)
    df = df.with_columns(
        (pl.col('close') / pl.col('close').shift(1) - 1).alias('returns')
    )
    
    if len(df) >= period:
        df = df.with_columns(
            pl.col('returns').rolling_std(window_size=period).alias('volatility')
        )
    
    return df.drop(['hl', 'hc', 'lc', 'true_range', 'returns'])


def add_volume_indicators(df: pl.DataFrame, period: int = 20) -> pl.DataFrame:
    """
    Calculate volume-based indicators.
    """
    if len(df) < period:
        return df.with_columns([
            pl.lit(None).alias('volume_sma'),
            pl.lit(None).alias('volume_ratio')
        ])
    
    df = df.with_columns(
        pl.col('volume').rolling_mean(window_size=period).alias('volume_sma')
    )
    
    df = df.with_columns(
        (pl.col('volume') / pl.col('volume_sma')).alias('volume_ratio')
    )
    
    return df


def add_price_action_signals(df: pl.DataFrame) -> pl.DataFrame:
    """
    Add price action signals and patterns.
    """
    if len(df) < 2:
        return df
    
    df = df.with_columns([
        # Daily price change
        (pl.col('close') - pl.col('open')).alias('candle_body'),
        (pl.col('high') - pl.col('low')).alias('candle_range'),
        
        # Price change percentage
        ((pl.col('close') - pl.col('close').shift(1)) / pl.col('close').shift(1) * 100).alias('pct_change'),
        
        # Higher highs and lower lows
        (pl.col('high') > pl.col('high').shift(1)).alias('higher_high'),
        (pl.col('low') < pl.col('low').shift(1)).alias('lower_low'),
        
        # Bullish/Bearish candle
        (pl.col('close') > pl.col('open')).alias('bullish_candle'),
    ])
    
    return df

print("Technical indicator functions defined successfully!")

## 4. Apply Technical Indicators to All Timeframes

Calculate comprehensive technical indicators for each timeframe.

In [ ]:
def enrich_with_indicators(df: pl.DataFrame, timeframe: str) -> pl.DataFrame:
    """
    Apply all technical indicators to a dataframe.
    """
    if df.is_empty():
        return df
    
    print(f"Calculating indicators for {timeframe}...")
    
    # Determine appropriate periods based on timeframe
    if timeframe in ['10min', '15min', '30min']:
        ma_periods = [5, 10, 20, 50]
        rsi_period = 14
        bb_period = 20
        vol_period = 14
    elif timeframe in ['1hour', '4hour']:
        ma_periods = [5, 10, 20, 50, 100]
        rsi_period = 14
        bb_period = 20
        vol_period = 14
    else:  # 1day
        ma_periods = [5, 10, 20, 50, 200]
        rsi_period = 14
        bb_period = 20
        vol_period = 20
    
    # Sort by datetime
    df = df.sort('datetime')
    
    # Apply indicators
    df = add_moving_averages(df, ma_periods)
    df = add_rsi(df, rsi_period)
    df = add_macd(df)
    df = add_bollinger_bands(df, bb_period)
    df = add_volatility_metrics(df, vol_period)
    df = add_volume_indicators(df, vol_period)
    df = add_price_action_signals(df)
    
    # Add timeframe identifier
    df = df.with_columns(pl.lit(timeframe).alias('timeframe'))
    
    print(f"  ✓ Completed {timeframe}: {len(df)} rows, {len(df.columns)} columns")
    return df

# Enrich all timeframes with indicators
enriched_data = {}

print("Enriching all timeframes with technical indicators...\n")
print("="*60)

for tf_name, df in timeframe_data.items():
    enriched_data[tf_name] = enrich_with_indicators(df, tf_name)

print("="*60)
print("\nIndicator calculation complete!")

## 5. Timeframe-Specific Analysis

### 5.1 - 10 Minute Analysis (Scalping Timeframe)

In [ ]:
df_10min = enriched_data['10min']

print("\n" + "="*60)
print("10-MINUTE TIMEFRAME ANALYSIS")
print("="*60)
print(f"Ticker: {PRIMARY_TICKER}")
print(f"Total candles: {len(df_10min)}")
print(f"Date range: {df_10min['datetime'].min()} to {df_10min['datetime'].max()}")

# Recent price action
print("\n📊 Latest 10 bars:")
print(df_10min.select([
    'datetime', 'open', 'high', 'low', 'close', 'volume',
    'sma_5', 'sma_10', 'rsi', 'pct_change'
]).tail(10))

# Key statistics
if not df_10min.is_empty():
    stats_10min = df_10min.select([
        pl.col('close').mean().alias('avg_price'),
        pl.col('close').std().alias('price_std'),
        pl.col('volume').mean().alias('avg_volume'),
        pl.col('rsi').mean().alias('avg_rsi'),
        pl.col('atr').mean().alias('avg_atr'),
        pl.col('pct_change').mean().alias('avg_pct_change'),
        pl.col('pct_change').std().alias('volatility')
    ])
    
    print("\n📈 10-Minute Statistics:")
    print(stats_10min)

# Identify high volatility periods
if 'volatility' in df_10min.columns:
    high_vol = df_10min.filter(
        pl.col('volatility') > pl.col('volatility').quantile(0.75)
    ).select(['datetime', 'close', 'volume', 'volatility', 'pct_change'])
    
    print(f"\n⚡ High Volatility Periods (Top 25%): {len(high_vol)} bars")
    print(high_vol.head(5))

# RSI extremes (oversold/overbought)
if 'rsi' in df_10min.columns:
    oversold = df_10min.filter(pl.col('rsi') < 30).select(
        ['datetime', 'close', 'rsi', 'volume']
    )
    overbought = df_10min.filter(pl.col('rsi') > 70).select(
        ['datetime', 'close', 'rsi', 'volume']
    )
    
    print(f"\n🔴 Oversold signals (RSI < 30): {len(oversold)}")
    if len(oversold) > 0:
        print(oversold.tail(3))
    
    print(f"\n🟢 Overbought signals (RSI > 70): {len(overbought)}")
    if len(overbought) > 0:
        print(overbought.tail(3))

### 5.2 - 15 Minute Analysis (Intraday Trading)

In [ ]:
df_15min = enriched_data['15min']

print("\n" + "="*60)
print("15-MINUTE TIMEFRAME ANALYSIS")
print("="*60)
print(f"Ticker: {PRIMARY_TICKER}")
print(f"Total candles: {len(df_15min)}")
print(f"Date range: {df_15min['datetime'].min()} to {df_15min['datetime'].max()}")

# Moving average crossovers
if 'sma_5' in df_15min.columns and 'sma_20' in df_15min.columns:
    df_15min_signals = df_15min.with_columns([
        (pl.col('sma_5') > pl.col('sma_20')).alias('bullish_ma_cross'),
        (pl.col('sma_5') < pl.col('sma_20')).alias('bearish_ma_cross')
    ])
    
    # Detect crossover points
    df_15min_signals = df_15min_signals.with_columns([
        (pl.col('bullish_ma_cross') & ~pl.col('bullish_ma_cross').shift(1)).alias('bullish_cross_signal'),
        (pl.col('bearish_ma_cross') & ~pl.col('bearish_ma_cross').shift(1)).alias('bearish_cross_signal')
    ])
    
    bullish_crosses = df_15min_signals.filter(pl.col('bullish_cross_signal'))
    bearish_crosses = df_15min_signals.filter(pl.col('bearish_cross_signal'))
    
    print(f"\n🔵 Bullish MA crossovers (5 > 20): {len(bullish_crosses)}")
    if len(bullish_crosses) > 0:
        print(bullish_crosses.select(['datetime', 'close', 'sma_5', 'sma_20', 'rsi']).tail(3))
    
    print(f"\n🔴 Bearish MA crossovers (5 < 20): {len(bearish_crosses)}")
    if len(bearish_crosses) > 0:
        print(bearish_crosses.select(['datetime', 'close', 'sma_5', 'sma_20', 'rsi']).tail(3))

# Volume analysis
if 'volume_ratio' in df_15min.columns:
    high_volume = df_15min.filter(
        pl.col('volume_ratio') > 1.5
    ).select(['datetime', 'close', 'volume', 'volume_ratio', 'pct_change'])
    
    print(f"\n📊 High Volume Bars (>1.5x average): {len(high_volume)}")
    if len(high_volume) > 0:
        print(high_volume.tail(5))

# MACD signals
if 'macd' in df_15min.columns and 'macd_signal' in df_15min.columns:
    df_15min_macd = df_15min.with_columns([
        (pl.col('macd') > pl.col('macd_signal')).alias('macd_bullish'),
        (pl.col('macd') < pl.col('macd_signal')).alias('macd_bearish')
    ])
    
    print("\n📈 MACD Analysis:")
    print(df_15min_macd.select([
        'datetime', 'close', 'macd', 'macd_signal', 'macd_histogram'
    ]).tail(10))

### 5.3 - 30 Minute Analysis (Short-term Trading)

In [ ]:
df_30min = enriched_data['30min']

print("\n" + "="*60)
print("30-MINUTE TIMEFRAME ANALYSIS")
print("="*60)
print(f"Ticker: {PRIMARY_TICKER}")
print(f"Total candles: {len(df_30min)}")
print(f"Date range: {df_30min['datetime'].min()} to {df_30min['datetime'].max()}")

# Bollinger Band analysis
if all(col in df_30min.columns for col in ['bb_upper', 'bb_lower', 'close']):
    df_30min_bb = df_30min.with_columns([
        (pl.col('close') > pl.col('bb_upper')).alias('above_bb_upper'),
        (pl.col('close') < pl.col('bb_lower')).alias('below_bb_lower'),
        ((pl.col('close') - pl.col('bb_lower')) / (pl.col('bb_upper') - pl.col('bb_lower'))).alias('bb_position')
    ])
    
    upper_touches = df_30min_bb.filter(pl.col('above_bb_upper'))
    lower_touches = df_30min_bb.filter(pl.col('below_bb_lower'))
    
    print(f"\n📊 Bollinger Band Analysis:")
    print(f"  Price above upper band: {len(upper_touches)} times")
    print(f"  Price below lower band: {len(lower_touches)} times")
    
    print("\n Latest BB positions:")
    print(df_30min_bb.select([
        'datetime', 'close', 'bb_upper', 'bb_middle', 'bb_lower', 'bb_position'
    ]).tail(10))

# Trend analysis using multiple moving averages
if all(col in df_30min.columns for col in ['sma_5', 'sma_10', 'sma_20']):
    df_30min_trend = df_30min.with_columns(
        pl.when(
            (pl.col('sma_5') > pl.col('sma_10')) & (pl.col('sma_10') > pl.col('sma_20'))
        ).then(pl.lit('strong_uptrend'))
        .when(
            (pl.col('sma_5') < pl.col('sma_10')) & (pl.col('sma_10') < pl.col('sma_20'))
        ).then(pl.lit('strong_downtrend'))
        .when(
            pl.col('sma_5') > pl.col('sma_20')
        ).then(pl.lit('uptrend'))
        .when(
            pl.col('sma_5') < pl.col('sma_20')
        ).then(pl.lit('downtrend'))
        .otherwise(pl.lit('sideways'))
        .alias('trend')
    )
    
    trend_dist = df_30min_trend.group_by('trend').agg(
        pl.count().alias('count')
    ).sort('count', descending=True)
    
    print("\n📈 Trend Distribution:")
    print(trend_dist)
    
    print("\n Current trend:")
    print(df_30min_trend.select(['datetime', 'close', 'trend', 'rsi', 'atr']).tail(5))

# Support and resistance levels (using rolling highs/lows)
if len(df_30min) > 20:
    df_30min_sr = df_30min.with_columns([
        pl.col('high').rolling_max(window_size=20).alias('resistance_20'),
        pl.col('low').rolling_min(window_size=20).alias('support_20')
    ])
    
    print("\n🎯 Support & Resistance Levels (20-period):")
    print(df_30min_sr.select([
        'datetime', 'close', 'support_20', 'resistance_20'
    ]).tail(5))

### 5.4 - 1 Hour Analysis (Swing Trading)

In [ ]:
df_1hour = enriched_data['1hour']

print("\n" + "="*60)
print("1-HOUR TIMEFRAME ANALYSIS")
print("="*60)
print(f"Ticker: {PRIMARY_TICKER}")
print(f"Total candles: {len(df_1hour)}")
print(f"Date range: {df_1hour['datetime'].min()} to {df_1hour['datetime'].max()}")

# Multi-timeframe momentum
if all(col in df_1hour.columns for col in ['sma_10', 'sma_20', 'sma_50']):
    df_1hour_momentum = df_1hour.with_columns([
        ((pl.col('close') - pl.col('sma_10')) / pl.col('sma_10') * 100).alias('distance_from_sma10'),
        ((pl.col('close') - pl.col('sma_20')) / pl.col('sma_20') * 100).alias('distance_from_sma20'),
        ((pl.col('close') - pl.col('sma_50')) / pl.col('sma_50') * 100).alias('distance_from_sma50')
    ])
    
    print("\n📊 Distance from Moving Averages:")
    print(df_1hour_momentum.select([
        'datetime', 'close', 'distance_from_sma10', 'distance_from_sma20', 'distance_from_sma50'
    ]).tail(10))

# ATR-based stop loss levels
if 'atr' in df_1hour.columns:
    df_1hour_stops = df_1hour.with_columns([
        (pl.col('close') - 2 * pl.col('atr')).alias('stop_loss_2atr'),
        (pl.col('close') + 2 * pl.col('atr')).alias('take_profit_2atr'),
        (pl.col('close') - 3 * pl.col('atr')).alias('stop_loss_3atr'),
        (pl.col('close') + 3 * pl.col('atr')).alias('take_profit_3atr')
    ])
    
    print("\n🎯 ATR-Based Trading Levels:")
    print(df_1hour_stops.select([
        'datetime', 'close', 'atr', 'stop_loss_2atr', 'take_profit_2atr'
    ]).tail(5))

# Divergence analysis (price vs RSI)
if 'rsi' in df_1hour.columns and len(df_1hour) > 5:
    df_1hour_div = df_1hour.with_columns([
        (pl.col('close') > pl.col('close').shift(5)).alias('price_higher'),
        (pl.col('rsi') > pl.col('rsi').shift(5)).alias('rsi_higher')
    ])
    
    # Bearish divergence: price makes higher high, RSI makes lower high
    bearish_div = df_1hour_div.filter(
        pl.col('price_higher') & ~pl.col('rsi_higher')
    )
    
    # Bullish divergence: price makes lower low, RSI makes higher low
    bullish_div = df_1hour_div.filter(
        ~pl.col('price_higher') & pl.col('rsi_higher')
    )
    
    print(f"\n⚠️ Divergence Signals:")
    print(f"  Bearish divergences: {len(bearish_div)}")
    print(f"  Bullish divergences: {len(bullish_div)}")

# Consolidation vs breakout identification
if 'atr' in df_1hour.columns:
    df_1hour_volatility = df_1hour.with_columns(
        (pl.col('atr') < pl.col('atr').rolling_mean(window_size=20)).alias('consolidation')
    )
    
    consolidating = df_1hour_volatility.filter(pl.col('consolidation'))
    
    print(f"\n📉 Market State:")
    print(f"  Consolidation periods: {len(consolidating)}")
    print(f"  Trending periods: {len(df_1hour) - len(consolidating)}")

### 5.5 - 4 Hour Analysis (Position Trading)

In [ ]:
df_4hour = enriched_data['4hour']

print("\n" + "="*60)
print("4-HOUR TIMEFRAME ANALYSIS")
print("="*60)
print(f"Ticker: {PRIMARY_TICKER}")
print(f"Total candles: {len(df_4hour)}")
print(f"Date range: {df_4hour['datetime'].min()} to {df_4hour['datetime'].max()}")

# Longer-term trend analysis
if all(col in df_4hour.columns for col in ['sma_20', 'sma_50']):
    current_price = df_4hour.select(pl.col('close').last()).item()
    sma_20 = df_4hour.select(pl.col('sma_20').last()).item()
    sma_50 = df_4hour.select(pl.col('sma_50').last()).item()
    
    print(f"\n📊 Current Technical Position:")
    print(f"  Current Price: ${current_price:.2f}")
    print(f"  20-period SMA: ${sma_20:.2f}")
    print(f"  50-period SMA: ${sma_50:.2f}")
    
    if current_price > sma_20 > sma_50:
        print("  Trend: STRONG BULLISH 🟢")
    elif current_price < sma_20 < sma_50:
        print("  Trend: STRONG BEARISH 🔴")
    elif current_price > sma_50:
        print("  Trend: BULLISH 🟢")
    else:
        print("  Trend: BEARISH 🔴")

# Price swings (for swing trading)
if len(df_4hour) > 10:
    df_4hour_swings = df_4hour.with_columns([
        pl.col('high').rolling_max(window_size=5).shift(1).alias('swing_high'),
        pl.col('low').rolling_min(window_size=5).shift(1).alias('swing_low')
    ])
    
    # Breakouts above swing highs
    breakouts = df_4hour_swings.filter(
        pl.col('close') > pl.col('swing_high')
    )
    
    # Breakdowns below swing lows
    breakdowns = df_4hour_swings.filter(
        pl.col('close') < pl.col('swing_low')
    )
    
    print(f"\n🚀 Swing Analysis:")
    print(f"  Breakouts (above swing high): {len(breakouts)}")
    print(f"  Breakdowns (below swing low): {len(breakdowns)}")
    
    if len(breakouts) > 0:
        print("\n  Recent breakouts:")
        print(breakouts.select(['datetime', 'close', 'swing_high', 'volume']).tail(3))

# Weekly statistics
if len(df_4hour) > 0:
    weekly_stats = df_4hour.select([
        pl.col('pct_change').mean().alias('avg_4h_change'),
        pl.col('pct_change').std().alias('volatility'),
        pl.col('volume').mean().alias('avg_volume'),
        pl.col('rsi').mean().alias('avg_rsi')
    ])
    
    print("\n📈 4-Hour Statistics:")
    print(weekly_stats)

### 5.6 - Daily Analysis (Investment/Long-term)

In [ ]:
df_1day = enriched_data['1day']

print("\n" + "="*60)
print("DAILY TIMEFRAME ANALYSIS")
print("="*60)
print(f"Ticker: {PRIMARY_TICKER}")
print(f"Total candles: {len(df_1day)}")
print(f"Date range: {df_1day['datetime'].min()} to {df_1day['datetime'].max()}")

# Major moving average analysis
if all(col in df_1day.columns for col in ['sma_50', 'sma_200']):
    current_price = df_1day.select(pl.col('close').last()).item()
    sma_50 = df_1day.select(pl.col('sma_50').last()).item()
    sma_200 = df_1day.select(pl.col('sma_200').last()).item()
    
    print(f"\n📊 Major Moving Averages:")
    print(f"  Current Price: ${current_price:.2f}")
    print(f"  50-day SMA: ${sma_50:.2f}")
    print(f"  200-day SMA: ${sma_200:.2f}")
    
    if sma_50 > sma_200:
        print("  Golden Cross: YES ✅ (Bullish)")
    else:
        print("  Death Cross: YES ❌ (Bearish)")
    
    # Detect recent crossovers
    df_1day_crosses = df_1day.with_columns([
        (pl.col('sma_50') > pl.col('sma_200')).alias('golden_cross_active'),
    ])
    
    df_1day_crosses = df_1day_crosses.with_columns(
        (pl.col('golden_cross_active') != pl.col('golden_cross_active').shift(1)).alias('cross_occurred')
    )
    
    recent_crosses = df_1day_crosses.filter(pl.col('cross_occurred')).tail(5)
    if len(recent_crosses) > 0:
        print("\n  Recent MA Crosses:")
        print(recent_crosses.select(['datetime', 'close', 'sma_50', 'sma_200']))

# Year-to-date performance
if len(df_1day) > 0:
    latest = df_1day.select('datetime', 'close').sort('datetime').tail(1)
    latest_date = latest['datetime'].item()
    latest_price = latest['close'].item()
    
    # Find first price of the year
    year_start = df_1day.filter(
        pl.col('datetime').dt.year() == latest_date.year
    ).sort('datetime').head(1)
    
    if len(year_start) > 0:
        year_start_price = year_start['close'].item()
        ytd_return = ((latest_price - year_start_price) / year_start_price) * 100
        
        print(f"\n📈 Year-to-Date Performance:")
        print(f"  Start of year price: ${year_start_price:.2f}")
        print(f"  Current price: ${latest_price:.2f}")
        print(f"  YTD Return: {ytd_return:+.2f}%")

# Monthly returns analysis
monthly_returns = df_1day.with_columns(
    pl.col('datetime').dt.strftime('%Y-%m').alias('month')
).group_by('month').agg([
    pl.col('close').first().alias('month_open'),
    pl.col('close').last().alias('month_close'),
    pl.col('high').max().alias('month_high'),
    pl.col('low').min().alias('month_low'),
    pl.col('volume').sum().alias('month_volume')
]).with_columns(
    ((pl.col('month_close') - pl.col('month_open')) / pl.col('month_open') * 100).alias('monthly_return')
).sort('month')

print("\n📅 Monthly Returns (Last 12 months):")
print(monthly_returns.tail(12))

# Statistical summary
if len(df_1day) > 0:
    daily_stats = df_1day.select([
        pl.col('close').max().alias('all_time_high'),
        pl.col('close').min().alias('all_time_low'),
        pl.col('pct_change').mean().alias('avg_daily_change'),
        pl.col('pct_change').std().alias('daily_volatility'),
        pl.col('volume').mean().alias('avg_daily_volume'),
        pl.col('rsi').mean().alias('avg_rsi')
    ])
    
    print("\n📊 Daily Statistics Summary:")
    print(daily_stats)

# Drawdown analysis
if len(df_1day) > 0:
    df_1day_dd = df_1day.with_columns(
        pl.col('close').cum_max().alias('running_max')
    ).with_columns(
        ((pl.col('close') - pl.col('running_max')) / pl.col('running_max') * 100).alias('drawdown_pct')
    )
    
    max_drawdown = df_1day_dd.select(pl.col('drawdown_pct').min()).item()
    max_dd_row = df_1day_dd.filter(pl.col('drawdown_pct') == max_drawdown)
    
    print(f"\n📉 Drawdown Analysis:")
    print(f"  Maximum Drawdown: {max_drawdown:.2f}%")
    if len(max_dd_row) > 0:
        print(f"  Occurred on: {max_dd_row['datetime'].item()}")
        print(f"  Price at max DD: ${max_dd_row['close'].item():.2f}")

## 6. Cross-Timeframe Analysis

Compare signals and trends across multiple timeframes for comprehensive market view.

In [ ]:
print("\n" + "="*60)
print("CROSS-TIMEFRAME ANALYSIS")
print("="*60)

# Get latest values from each timeframe
def get_latest_metrics(df: pl.DataFrame, timeframe: str) -> dict:
    """Extract latest metrics from a timeframe."""
    if df.is_empty():
        return None
    
    latest = df.sort('datetime').tail(1)
    
    metrics = {
        'timeframe': timeframe,
        'datetime': latest['datetime'].item(),
        'close': latest['close'].item(),
    }
    
    # Add available indicators
    for col in ['rsi', 'macd', 'macd_signal', 'atr', 'bb_width', 'volume_ratio']:
        if col in latest.columns:
            metrics[col] = latest[col].item()
    
    # Trend determination
    if 'sma_5' in latest.columns and 'sma_20' in latest.columns:
        sma_5 = latest['sma_5'].item()
        sma_20 = latest['sma_20'].item()
        close = latest['close'].item()
        
        if close > sma_5 > sma_20:
            metrics['trend'] = 'bullish'
        elif close < sma_5 < sma_20:
            metrics['trend'] = 'bearish'
        else:
            metrics['trend'] = 'mixed'
    
    return metrics

# Collect metrics from all timeframes
timeframe_comparison = []
for tf_name, df in enriched_data.items():
    metrics = get_latest_metrics(df, tf_name)
    if metrics:
        timeframe_comparison.append(metrics)

# Create comparison dataframe
if timeframe_comparison:
    df_comparison = pl.DataFrame(timeframe_comparison)
    
    print("\n📊 Latest Metrics Across All Timeframes:")
    print(df_comparison)
    
    # Trend alignment
    if 'trend' in df_comparison.columns:
        trend_summary = df_comparison.group_by('trend').agg(
            pl.col('timeframe').count().alias('count')
        )
        
        print("\n🎯 Trend Alignment:")
        print(trend_summary)
        
        bullish_count = trend_summary.filter(pl.col('trend') == 'bullish')['count'].sum()
        bearish_count = trend_summary.filter(pl.col('trend') == 'bearish')['count'].sum()
        
        print(f"\n  Overall Market Bias:")
        if bullish_count > bearish_count:
            print(f"  BULLISH ({bullish_count}/{len(timeframe_comparison)} timeframes) 🟢")
        elif bearish_count > bullish_count:
            print(f"  BEARISH ({bearish_count}/{len(timeframe_comparison)} timeframes) 🔴")
        else:
            print(f"  NEUTRAL (Mixed signals) ⚪")
    
    # RSI comparison
    if 'rsi' in df_comparison.columns:
        print("\n📊 RSI Across Timeframes:")
        rsi_data = df_comparison.select(['timeframe', 'rsi']).sort('timeframe')
        print(rsi_data)
        
        avg_rsi = df_comparison['rsi'].mean()
        print(f"\n  Average RSI across all timeframes: {avg_rsi:.2f}")
        
        if avg_rsi < 30:
            print("  ⚠️ Overall OVERSOLD condition")
        elif avg_rsi > 70:
            print("  ⚠️ Overall OVERBOUGHT condition")
        else:
            print("  ✅ Neutral RSI conditions")

print("\n" + "="*60)

## 7. Trading Signals Summary

Consolidated view of trading signals across all timeframes.

In [ ]:
def generate_trading_signals(df: pl.DataFrame, timeframe: str) -> dict:
    """
    Generate trading signals based on multiple indicators.
    """
    if df.is_empty() or len(df) < 5:
        return None
    
    latest = df.sort('datetime').tail(1)
    
    signals = {
        'timeframe': timeframe,
        'datetime': latest['datetime'].item(),
        'price': latest['close'].item()
    }
    
    bullish_signals = 0
    bearish_signals = 0
    signal_details = []
    
    # RSI signals
    if 'rsi' in latest.columns:
        rsi = latest['rsi'].item()
        if rsi is not None:
            if rsi < 30:
                bullish_signals += 1
                signal_details.append(f"RSI oversold ({rsi:.1f})")
            elif rsi > 70:
                bearish_signals += 1
                signal_details.append(f"RSI overbought ({rsi:.1f})")
    
    # MACD signals
    if 'macd' in latest.columns and 'macd_signal' in latest.columns:
        macd = latest['macd'].item()
        macd_signal = latest['macd_signal'].item()
        if macd is not None and macd_signal is not None:
            if macd > macd_signal:
                bullish_signals += 1
                signal_details.append("MACD bullish")
            else:
                bearish_signals += 1
                signal_details.append("MACD bearish")
    
    # Moving average signals
    if 'sma_5' in latest.columns and 'sma_20' in latest.columns:
        close = latest['close'].item()
        sma_5 = latest['sma_5'].item()
        sma_20 = latest['sma_20'].item()
        
        if sma_5 is not None and sma_20 is not None:
            if close > sma_5 > sma_20:
                bullish_signals += 1
                signal_details.append("MA alignment bullish")
            elif close < sma_5 < sma_20:
                bearish_signals += 1
                signal_details.append("MA alignment bearish")
    
    # Bollinger Band signals
    if all(col in latest.columns for col in ['close', 'bb_upper', 'bb_lower']):
        close = latest['close'].item()
        bb_upper = latest['bb_upper'].item()
        bb_lower = latest['bb_lower'].item()
        
        if all(v is not None for v in [close, bb_upper, bb_lower]):
            if close < bb_lower:
                bullish_signals += 1
                signal_details.append("Below BB lower band")
            elif close > bb_upper:
                bearish_signals += 1
                signal_details.append("Above BB upper band")
    
    # Determine overall signal
    total_signals = bullish_signals + bearish_signals
    if total_signals > 0:
        if bullish_signals > bearish_signals:
            signals['signal'] = 'BUY'
            signals['strength'] = f"{bullish_signals}/{total_signals}"
        elif bearish_signals > bullish_signals:
            signals['signal'] = 'SELL'
            signals['strength'] = f"{bearish_signals}/{total_signals}"
        else:
            signals['signal'] = 'HOLD'
            signals['strength'] = f"{max(bullish_signals, bearish_signals)}/{total_signals}"
    else:
        signals['signal'] = 'HOLD'
        signals['strength'] = '0/0'
    
    signals['details'] = ', '.join(signal_details) if signal_details else 'No clear signals'
    
    return signals

print("\n" + "="*60)
print("TRADING SIGNALS SUMMARY")
print("="*60)

# Generate signals for all timeframes
all_signals = []
for tf_name, df in enriched_data.items():
    signals = generate_trading_signals(df, tf_name)
    if signals:
        all_signals.append(signals)

if all_signals:
    df_signals = pl.DataFrame(all_signals)
    
    print("\n🎯 Trading Signals Across All Timeframes:")
    print(df_signals.select(['timeframe', 'signal', 'strength', 'details']))
    
    # Signal consensus
    signal_counts = df_signals.group_by('signal').agg(
        pl.col('timeframe').count().alias('count')
    ).sort('count', descending=True)
    
    print("\n📊 Signal Consensus:")
    print(signal_counts)
    
    # Overall recommendation
    buy_count = signal_counts.filter(pl.col('signal') == 'BUY')['count'].sum()
    sell_count = signal_counts.filter(pl.col('signal') == 'SELL')['count'].sum()
    hold_count = signal_counts.filter(pl.col('signal') == 'HOLD')['count'].sum()
    
    print("\n" + "="*60)
    print("OVERALL RECOMMENDATION")
    print("="*60)
    
    if buy_count > sell_count and buy_count > hold_count:
        print(f"\n🟢 BULLISH BIAS: {buy_count}/{len(all_signals)} timeframes suggest BUY")
    elif sell_count > buy_count and sell_count > hold_count:
        print(f"\n🔴 BEARISH BIAS: {sell_count}/{len(all_signals)} timeframes suggest SELL")
    else:
        print(f"\n⚪ NEUTRAL: Mixed signals across timeframes (Hold recommended)")
    
    print("\n⚠️ Disclaimer: This is for educational purposes only.")
    print("   Not financial advice. Always do your own research.")

print("\n" + "="*60)

## 8. Export Data for Further Analysis

Save processed data to CSV files for external analysis.

In [ ]:
# Uncomment to export data

# for tf_name, df in enriched_data.items():
#     if not df.is_empty():
#         filename = f"{PRIMARY_TICKER}_{tf_name}_analysis.csv"
#         df.write_csv(filename)
#         print(f"Exported {filename}")

print("\n✅ Analysis Complete!")
print("\nTo export data, uncomment the code block above and run the cell.")

## Summary

This notebook demonstrates:

1. **Data Acquisition**: Fetching multi-timeframe stock data using yfinance
2. **Polars Integration**: Converting and processing data with Polars for high performance
3. **Technical Indicators**: Implementing various indicators (MA, RSI, MACD, BB, ATR, etc.)
4. **Multi-Timeframe Analysis**: Analyzing 10min, 15min, 30min, 1hr, 4hr, and daily timeframes
5. **Cross-Timeframe Comparison**: Identifying trends and signals across different timeframes
6. **Trading Signals**: Generating actionable signals based on multiple indicators
7. **Polars Features**: Leveraging expressions, lazy evaluation, window functions, and aggregations

### Key Polars Operations Used:
- DataFrame transformations with `with_columns`
- Window functions with `rolling_mean`, `rolling_std`, `rolling_max`, `rolling_min`
- Conditional expressions with `when`, `then`, `otherwise`
- Aggregations with `group_by` and `agg`
- Filtering and sorting
- DateTime operations
- Expression chaining for complex calculations

This approach provides a comprehensive framework for stock market analysis using modern data tools.